# Synkro + Liquid AI Models (Fully Local)

Generate training datasets using Liquid AI models — **100% local, no API keys needed**:
- **LFM2-350M** (Ollama) — fast generation
- **LFM2-1.2B** (Ollama) — validation and repairs

**Before running:**
1. Runtime > Change runtime type > GPU (T4)

This notebook:
1. Installs Ollama and synkro
2. Downloads both Liquid models for local inference
3. Runs synkro's pipeline with Ollama structured output
4. Saves the training dataset

In [1]:
# Install synkro and Ollama
import os
import shutil

%pip install -q --upgrade --no-cache-dir synkro
%system sudo apt-get update && sudo apt-get install -y zstd
%system curl -fsSL https://ollama.ai/install.sh | sh

import synkro
print("\n" + "=" * 60)
print(f"Synkro v{synkro.__version__} installed!")
print("=" * 60)

ollama_path = shutil.which("ollama")
if ollama_path is None:
    for p in ["/usr/local/bin/ollama", os.path.expanduser("~/.local/bin/ollama")]:
        if os.path.exists(p):
            ollama_path = p
            break

if ollama_path:
    print(f"Ollama found at: {ollama_path}")
else:
    print("ERROR: Ollama not found. Ensure GPU runtime is enabled.")
    raise RuntimeError("Ollama installation failed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.3/314.3 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 198.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 209.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 210.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 261.5 MB/s eta 0:00:00

Synkro v0.5.47 installed!
Ollama found at: /usr/local/bin/ollama


In [2]:
# Start Ollama server in background
import os
import shutil
import subprocess
import time

ollama_path = shutil.which("ollama")
if ollama_path is None:
    for p in ["/usr/local/bin/ollama", os.path.expanduser("~/.local/bin/ollama")]:
        if os.path.exists(p):
            ollama_path = p
            break

if ollama_path:
    print(f"Starting Ollama server from: {ollama_path}")
    process = subprocess.Popen(
        [ollama_path, "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
    )
    time.sleep(5)
    print("Ollama server running!")
else:
    raise RuntimeError("Ollama not found. Run installation cell first.")

Starting Ollama server from: /usr/local/bin/ollama
Ollama server running!


In [3]:
# Pull both Liquid models and preload to GPU
import httpx

MODEL_GEN = "hf.co/LiquidAI/LFM2-350M-GGUF"
MODEL_VAL = "hf.co/LiquidAI/LFM2-1.2B-GGUF"

print("Pulling LFM2-350M (generation)...")
%system ollama pull {MODEL_GEN}

print("\nPulling LFM2-1.2B (validation)...")
%system ollama pull {MODEL_VAL}

print("\n" + "=" * 60)
print("Models downloaded! Now preloading to GPU...")
print("=" * 60)

for name, model in [("LFM2-350M", MODEL_GEN), ("LFM2-1.2B", MODEL_VAL)]:
    try:
        with httpx.Client(timeout=180) as client:
            response = client.post(
                "http://localhost:11434/api/generate",
                json={
                    "model": model,
                    "prompt": "",
                    "stream": False,
                    "keep_alive": "30m"
                }
            )
            response.raise_for_status()
        print(f"{name} preloaded to GPU!")
    except Exception as e:
        print(f"{name} preload failed (will load on first use): {e}")

%system ollama list

Pulling LFM2-350M (generation)...

Pulling LFM2-1.2B (validation)...

Models downloaded! Now preloading to GPU...
LFM2-350M preloaded to GPU!
LFM2-1.2B preloaded to GPU!


['NAME                                    ID              SIZE      MODIFIED       ',
 'hf.co/LiquidAI/LFM2-1.2B-GGUF:latest    0a0b3b829b72    730 MB    3 seconds ago     ',
 'hf.co/LiquidAI/LFM2-350M-GGUF:latest    f9037c479270    229 MB    14 seconds ago    ']

In [4]:
# Verify both models work
import httpx

MODEL_GEN = "hf.co/LiquidAI/LFM2-350M-GGUF"
MODEL_VAL = "hf.co/LiquidAI/LFM2-1.2B-GGUF"
OLLAMA_URL = "http://localhost:11434"

for name, model in [("LFM2-350M (gen)", MODEL_GEN), ("LFM2-1.2B (val)", MODEL_VAL)]:
    print(f"Testing {name}...")
    try:
        with httpx.Client(timeout=30) as client:
            response = client.post(
                f"{OLLAMA_URL}/api/generate",
                json={
                    "model": model,
                    "prompt": "Say hello",
                    "stream": False,
                    "keep_alive": "30m"
                }
            )
            response.raise_for_status()
            data = response.json()
            print(f"  Response: {data.get('response', 'NO RESPONSE')[:80]}")
    except Exception as e:
        print(f"  ERROR: {e}")
        raise

print("-" * 40)
print("Both models working!")

Testing LFM2-350M (gen)...
  Response: Hi there! How can I assist you today?
Testing LFM2-1.2B (val)...
  Response: Hello! How can I assist you today?
----------------------------------------
Both models working!


In [5]:
# Define test policy (same as quickstart.py)
EXPENSE_POLICY = """
# Company Expense Policy

## Approval Requirements
- All purchases over $50 require manager approval
- Travel expenses over $500 require VP approval
- Employees cannot approve their own expenses

## Refund Policy
- Refunds must be processed within 7 business days
- Refunds over $100 require supervisor approval

## Documentation
- All expenses require receipts
- Missing receipts void the expense claim
"""

print(f"Policy loaded! ({len(EXPENSE_POLICY)} chars)")

Policy loaded! (405 chars)


In [6]:
# Create synkro pipeline with Liquid AI models (both local)
from synkro import Local, create_pipeline
from synkro.types import DatasetType

MODEL_GEN = "hf.co/LiquidAI/LFM2-350M-GGUF"
MODEL_VAL = "hf.co/LiquidAI/LFM2-1.2B-GGUF"

pipeline = create_pipeline(
    model=Local.OLLAMA(MODEL_GEN),           # Generation (350M - local)
    grading_model=Local.OLLAMA(MODEL_VAL),   # Validation (1.2B - local)
    dataset_type=DatasetType.CONVERSATION,
    skip_grading=True,
    enable_hitl=False,
    temperature=0.0,
)

print("Pipeline created!")
print(f"Generation: {MODEL_GEN} (Ollama - local)")
print(f"Validation: {MODEL_VAL} (Ollama - local)")
print("No API keys needed!")

Pipeline created!
Generation: hf.co/LiquidAI/LFM2-350M-GGUF (Ollama - local)
Validation: hf.co/LiquidAI/LFM2-1.2B-GGUF (Ollama - local)
No API keys needed!


In [7]:
# Generate dataset
import nest_asyncio
nest_asyncio.apply()

print("=" * 60)
print("GENERATING DATASET")
print("=" * 60)

try:
    dataset = pipeline.generate(EXPENSE_POLICY, traces=5)

    print(f"\nGenerated {len(dataset)} traces!")

    if len(dataset) > 0:
        trace = dataset[0]
        print(f"\nFirst trace ({len(trace.messages)} messages):")
        for msg in trace.messages[:4]:
            role = msg.role.upper()
            content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
            print(f"   [{role}] {content}")

except Exception as e:
    print(f"\nGeneration failed: {e}")
    import traceback
    traceback.print_exc()
    dataset = None

Output()

GENERATING DATASET


╭──────────────────────────── SYNKRO  │  LFM2-350M-GGUF  │  conversation  │  5 traces ────────────────────────────╮
│                                                                                                                 │
│ Generated 5 traces in 2m 20s                                                                                    │
│                                                                                                                 │
│ ├─ 11 rules extracted                                                                                           │
│ ├─ 5 scenarios (4+ 1- 0! 0o)                                                                                    │
│ ├─ 5 traces synthesized                                                                                         │
│ └─ Cost: $0.00000000                                                                                            │
│                                                                                                                 │
╰────────────────────────────── ✓ Complete  │  5 traces  │  2m 20s  │  $0.00000000 ───────────────────────────────╯


Generated 5 traces!

First trace (5 messages):
   [USER] I need to submit an expense for a client lunch for $180, but the purchase date i...
   [ASSISTANT] Okay, let's break this down. First, you need to submit an expense for a client l...
   [USER] I have a digital receipt, but I don't have a purchase date.  I need manager appr...
   [ASSISTANT] Understood. Let's start with the rules.  Since you don't have a purchase date, w...


In [ ]:
# Save dataset to JSONL and preview
import json

if dataset and len(dataset) > 0:
    output_path = "liquid_traces.jsonl"
    dataset.save(output_path, pretty_print=False)
    print(f"Saved {len(dataset)} traces to {output_path}")
    print("=" * 60)

    with open(output_path, "r") as f:
        for i, line in enumerate(f, 1):
            trace_data = json.loads(line.strip())
            msgs = trace_data.get("messages", [])
            print(f"\n--- Trace {i} ({len(msgs)} messages) ---")
            for msg in msgs[:4]:
                role = msg.get("role", "?").upper()
                content = msg.get("content", "")
                if len(content) > 100:
                    content = content[:100] + "..."
                print(f"  [{role}] {content}")
            if len(msgs) > 4:
                print(f"  ... +{len(msgs) - 4} more messages")
else:
    print("No dataset to save")

In [9]:
# Summary
print("=" * 60)
print("SUMMARY")
print("=" * 60)

if dataset and len(dataset) > 0:
    print(f"SUCCESS - Generated {len(dataset)} training traces")
    print(f"Generation: LFM2-350M (Ollama - local)")
    print(f"Validation: LFM2-1.2B (Ollama - local)")
    print("Output: liquid_traces.jsonl")
else:
    print("FAILED - Check error messages above")
    print("\nTroubleshooting:")
    print("  1. Ensure GPU runtime: Runtime > Change runtime type > GPU")
    print("  2. Check Ollama: %system ps aux | grep ollama")
    print("  3. Check models: %system ollama list")

SUMMARY
SUCCESS - Generated 5 training traces
Generation: LFM2-350M (Ollama - local)
Validation: LFM2-1.2B (Ollama - local)
Output: liquid_traces.jsonl
